# Build and Test a Mini RAG System from Scratch 🧠

In [ ]:
!pip install sentence-transformers transformers torch

## ⚙️ Part 1: The Retriever - Finding the Right Knowledge

First, we'll set up our Retriever. Its job is to take a question and find the most relevant piece of text from our knowledge base.

1.  **Load the Embedding Model** (`all-MiniLM-L6-v2`)
2.  **Create our Knowledge Base**
3.  **Encode Everything into Embeddings**
4.  **Calculate Similarity** to find the best match

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

print("✅ Libraries imported successfully!")

# 1. Load our embedding model
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Create a simple knowledge base
knowledge_base = [
    "The capital of France is Paris, a city famous for the Eiffel Tower and the Louvre museum.",
    "The Amazon rainforest is the world's largest tropical rainforest, known for its incredible biodiversity.",
    "Mount Everest is the highest mountain on Earth, located in the Himalayas.",
    "The Great Wall of China is a series of fortifications stretching over 13,000 miles.",
    "Photosynthesis is the process used by plants to convert light energy into chemical energy."
]

# 3. Encode our knowledge base into embeddings
knowledge_embeddings = retriever_model.encode(knowledge_base, convert_to_tensor=True)

print(f"✅ Retriever model loaded and knowledge base encoded with {len(knowledge_base)} documents.")

## ✍️ Part 2: The Generator - Extracting the Answer

Now we set up our Generator. This model will take the question and the context found by the retriever and extract the exact answer from it.

In [ ]:
# Load our question-answering (generator) model
generator = pipeline('question-answering', model='distilbert-base-cased-distilled-squad')

print("✅ Generator (QA) model loaded.")

## 🚀 Part 3: Testing our RAG System

Time to put it all together! The function below will simulate a full RAG pipeline and grade itself against a predefined set of questions and answers.

It will test two key things:
1.  **Retrieval Accuracy**: Did we find the right document?
2.  **Generation Accuracy**: Did we extract the correct answer from that document?

In [ ]:
def run_rag_assessment():
    """Runs a self-assessment of the RAG pipeline with multiple questions."""

    # Define our questions, expected context keywords, and expected answers
    test_questions = [
        {
            "question": "What is the highest mountain?",
            "expected_keyword": "Everest",
            "expected_answer": "Mount Everest"
        },
        {
            "question": "Which city is home to the Louvre museum?",
            "expected_keyword": "France",
            "expected_answer": "Paris"
        },
        {
            "question": "What process do plants use for energy?",
            "expected_keyword": "Photosynthesis",
            "expected_answer": "Photosynthesis"
        },
    ]

    score = 0
    total = len(test_questions) * 2 # 2 points per question (1 for retrieval, 1 for generation)

    print("--- 🚀 Starting RAG System Assessment ---\n")

    for i, test in enumerate(test_questions):
        question = test["question"]
        print(f"\n--- Question {i+1}: '{question}' ---")

        # --- 1. Retrieval Step ---
        question_embedding = retriever_model.encode(question, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(question_embedding, knowledge_embeddings)[0]
        top_result_index = torch.argmax(cos_scores)
        retrieved_context = knowledge_base[top_result_index]

        print(f"🔎  Retrieved Context: '{retrieved_context}'")

        # Check if the retrieval was correct
        if test["expected_keyword"] in retrieved_context:
            print("✅  Retrieval Correct!")
            score += 1
        else:
            print(f"❌  Retrieval Failed. Expected context with keyword: '{test['expected_keyword']}'")

        # --- 2. Generation Step ---
        qa_result = generator(question=question, context=retrieved_context)
        generated_answer = qa_result['answer']

        print(f"✍️  Generated Answer: '{generated_answer}'")

        # Check if the generation was correct
        if test["expected_answer"].lower() in generated_answer.lower():
            print("✅  Generation Correct!")
            score += 1
        else:
            print(f"❌  Generation Failed. Expected answer: '{test['expected_answer']}'")

    # --- Final Score ---
    print(f"\n--- 🏁 Assessment Complete ---")
    print(f"🎯 Final Score: {score} / {total}")
    if score == total:
        print("🎉🎉🎉 Perfect! Our RAG system is working as expected!")
    elif score >= total / 2:
        print("👍 Good job! The system is mostly correct.")
    else:
        print("🔧 The system ran into some issues")

# Run the assessment!
run_rag_assessment()

# TASKS 🧑‍💻

### Task 1 (Challenge): Add a New Question

The task is to test the system with a new question about the **existing knowledge**.

In [ ]:
# Task 2: Adding new question to the assessment function


def run_rag_assessment_task_2():
    test_questions = [
        {
            "question": "What is the world's largest tropical rainforest?",
            "expected_keyword": "Amazon",
            "expected_answer": "Amazon rainforest"
        },
        {
            "question": "How long is the Great Wall of China?",
            "expected_keyword": "Great Wall of China",
            "expected_answer": "13,000 miles"
        },
        {
            "question": "Where is Mount Everest located?",
            "expected_keyword": "Himalayas",
            "expected_answer": "Himalayas"
        },
        {
            "question": "Which museum is located in Paris and is famous worldwide?",
            "expected_keyword": "Louvre",
            "expected_answer": "Louvre museum"
        }
    ]

    score = 0
    total = len(test_questions) * 2

    print("--- 🚀 Starting RAG System Assessment (Task 2) ---\n")

    for i, test in enumerate(test_questions):
        question = test["question"]
        print(f"\n--- Question {i+1}: '{question}' ---")
        question_embedding = retriever_model.encode(question, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(question_embedding, knowledge_embeddings)[0]
        top_result_index = torch.argmax(cos_scores)
        retrieved_context = knowledge_base[top_result_index]
        print(f"🔎  Retrieved Context: '{retrieved_context}'")
        if test["expected_keyword"] in retrieved_context:
            print("✅  Retrieval Correct!")
            score += 1
        else:
            print(f"❌  Retrieval Failed. Expected context with keyword: '{test['expected_keyword']}'")
        qa_result = generator(question=question, context=retrieved_context)
        generated_answer = qa_result['answer']
        print(f"✍️  Generated Answer: '{generated_answer}'")
        if test["expected_answer"].lower() in generated_answer.lower():
            print("✅  Generation Correct!")
            score += 1
        else:
            print(f"❌  Generation Failed. Expected answer: '{test['expected_answer']}'")

    print(f"\n--- 🏁 Assessment Complete ---")
    print(f"🎯 Final Score: {score} / {total}")
    if score == total:
        print("🎉🎉🎉 Perfect! O RAG system handled the new question!")

# Run the updated assessment
run_rag_assessment_task_2()

### Task 3 (Advanced Challenge): Add New Knowledge & Test It

Your final and most important task is to **expand the RAG system's knowledge base** and then test it.

**Instructions:**
1.  **Add a new fact** to the `knowledge_base` in the code cell below.
2.  **You must re-run this cell** to update the `knowledge_embeddings`! The system won't know about the new fact until you do.
3.  Finally, run the last code cell, which has a new test question about the knowledge you just added.

In [ ]:
# Task 3, Step 1: Add a new sentence to the knowledge base

knowledge_base_task_3 = [
    "The Sahara Desert is the largest hot desert in the world, located in North Africa."
]

# Re-encode the updated knowledge base
knowledge_embeddings_task_3 = retriever_model.encode(knowledge_base_task_3, convert_to_tensor=True)

print(f"✅ Knowledge base updated and re-encoded with {len(knowledge_base_task_3)} documents.")

In [ ]:
# Task 3, Step 2: Test your newly added knowledge

def run_rag_assessment_task_3():
    test_questions = [
        # --- NEW QUESTION FOR YOUR NEW KNOWLEDGE ---
        {
            "question": "What is the largest hot desert in the world?",
            "expected_keyword": "Sahara Desert",
            "expected_answer": "The Sahara Desert"
        }
    ]

    score = 0
    total = len(test_questions) * 2

    print("--- 🚀 Starting RAG System Assessment (Task 3) ---\n")

    for i, test in enumerate(test_questions):
        question = test["question"]
        print(f"\n--- Question {i+1}: '{question}' ---")
        # Use the updated embeddings from Task 3
        question_embedding = retriever_model.encode(question, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(question_embedding, knowledge_embeddings_task_3)[0]
        # Use the updated knowledge base from Task 3
        top_result_index = torch.argmax(cos_scores)
        retrieved_context = knowledge_base_task_3[top_result_index]
        print(f"🔎  Retrieved Context: '{retrieved_context}'")
        if test["expected_keyword"] in retrieved_context:
            print("✅  Retrieval Correct!")
            score += 1
        else:
            print(f"❌  Retrieval Failed. Expected context with keyword: '{test['expected_keyword']}'")
        qa_result = generator(question=question, context=retrieved_context)
        generated_answer = qa_result['answer']
        print(f"✍️  Generated Answer: '{generated_answer}'")
        if test["expected_answer"].lower() in generated_answer.lower():
            print("✅  Generation Correct!")
            score += 1
        else:
            print(f"❌  Generation Failed. Expected answer: '{test['expected_answer']}'")

    print(f"\n--- 🏁 Assessment Complete ---")
    print(f"🎯 Final Score: {score} / {total}")
    if score == total:
        print("🏆🏆🏆 Success! You have successfully extended the knowledge of your RAG system!")

# Run the final assessment
run_rag_assessment_task_3()